In [1]:
import numpy as np
import pandas as pd
import requests

In [41]:
sani = pd.read_csv('Data/311_Sanitation_2017.csv')
vacant = pd.read_csv('Data/311_Vacant_and_Abandoned_Buildings_2017.csv')
comm = pd.read_csv('Data/CommAreas.csv')

In [42]:
sani_lat = sani.Latitude.tolist()
sani_long = sani.Longitude.tolist()
sani_FIPS = []

In [58]:
for i in range(len(sani_lat)):
    lat = sani_lat[i]
    long = sani_long[i]
    url = 'https://www.broadbandmap.gov/broadbandmap/census/block?latitude={}&longitude={}&format=json'.format(lat, long)
    r = requests.get(url)
    rv = r.json()
    sani_FIPS.append(rv['Results']['block'][0]['FIPS'])

In [45]:
vacant_lat = vacant.LATITUDE.tolist()
vacant_long = vacant.LONGITUDE.tolist()
vacant_FIPS = []

In [46]:
for i in range(len(vacant_lat)):
    lat = vacant_lat[i]
    long = vacant_long[i]
    url = 'https://www.broadbandmap.gov/broadbandmap/census/block?latitude={}&longitude={}&format=json'.format(lat, long)
    r = requests.get(url)
    try:
        rv = r.json()
    except: 
        rv = 'null'
    if rv == 'null':
        vacant_FIPS.append(rv)
    else:
        vacant_FIPS.append(rv['Results']['block'][0]['FIPS'])

In [50]:
vacant['FIPS']= vacant_FIPS

In [53]:
vacant.to_csv('vacant')

In [59]:
sani['FIPS']=sani_FIPS

In [60]:
sani.to_csv('sani.csv')

In [23]:
sani_fips = pd.read_csv('Data/sani_2017_fips.csv')
vacant_fips = pd.read_csv('Data/vacant_2017_fips.csv')

In [24]:
all_fips = sani_fips.FIPS.append(vacant_fips.FIPS).apply(str)

In [41]:
fips = all_fips.to_frame()
fips['State'] = fips.FIPS.apply(lambda x: x[:2])
fips['County'] = fips.FIPS.apply(lambda x: x[2:5])
fips['Track'] = fips.FIPS.apply(lambda x: x[5:11])

In [42]:
fips_unique_track = fips.drop('FIPS',1)

In [43]:
fips_unique_track.drop_duplicates('Track', inplace=True)

In [48]:
fips_unique_track = fips_unique_track.reset_index(drop=True)

In [50]:
all_track = fips_unique_track.Track.tolist()

In [57]:
income = []
familysize = []
population = []
black = []

In [58]:
for i in range(len(all_track)):
    url = 'http://api.census.gov/data/2015/acs5/profile?get=DP03_0063E,DP02_0016E,DP05_0028E,DP05_0033E&for=tract:{}&in=state:17+county:031&key=d133f1c21f4fb823971ad5b8d7435f0997aa8986'.format(all_track[i])
    r = requests.get(url)
    rv = r.json()
    income.append(rv[1][0])
    familysize.append(rv[1][1])
    population.append(rv[1][2])
    black.append(rv[1][3])

In [25]:
sani_fips.FIPS = sani_fips.FIPS.apply(str)
sani_fips['Track'] = sani_fips.FIPS.apply(lambda x: x[5:11]).astype("int")
vacant_fips.FIPS = vacant_fips.FIPS.apply(str)
vacant_fips['Track'] = vacant_fips.FIPS.apply(lambda x: x[5:11]).astype("int")

In [ ]:
fips_unique_track['Mean Income'] = income
fips_unique_track['Family Size'] = familysize
fips_unique_track['population'] = population
fips_unique_track['black'] = black
fips_unique_track.to_csv('track_demographic.csv')

In [26]:
track_data = pd.read_csv('Data/track_demographic.csv')

In [21]:
sani_fips = sani_fips.join(track_data, on='Track', rsuffix='0')

In [28]:
sani_demo = pd.merge(sani_fips, track_data, on='Track')


,Unnamed: 0_x,Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,What is the Nature of this Code Violation?,Street Address,ZIP Code,X Coordinate,...,Location,FIPS,Track,Unnamed: 0_y,State,County,Mean Income,Family Size,Population,Black
0,0,1/2/17,Completed,1/3/17,17-00012728,Sanitation Code Violation,Garbage in alley,6156 S KARLOV AVE,60629.0,1150093.172,...,"(41.78123782345127, -87.72527525417806)",170316502002004,650200,0,17,31,58911,4.21,7653,98
1,1268,2/2/17,Completed,2/7/17,17-00613954,Sanitation Code Violation,Other,6205 S KOLIN AVE,60629.0,1148437.466,...,"(41.78028698735843, -87.73135471074477)",170316502003012,650200,0,17,31,58911,4.21,7653,98
2,1373,2/6/17,Completed,2/7/17,17-00683272,Sanitation Code Violation,Overflowing carts,6151 S KEELER AVE,60629.0,1149425.843,...,"(41.78130333820584, -87.7277213688664)",170316502002008,650200,0,17,31,58911,4.21,7653,98
3,1374,2/6/17,Completed,2/7/17,17-00683477,Sanitation Code Violation,Overflowing carts,5952 S PULASKI RD,60629.0,1150723.875,...,"(41.78475601976906, -87.72292944945652)",170316502001005,650200,0,17,31,58911,4.21,7653,98
4,1375,2/6/17,Completed,2/7/17,17-00683576,Sanitation Code Violation,Overflowing carts,5956 S PULASKI RD,60629.0,1150724.375,...,"(41.78470140133405, -87.72292813419426)",170316502001005,650200,0,17,31,58911,4.21,7653,98
5,1426,2/7/17,Completed,2/7/17,17-00684659,Sanitation Code Violation,Overflowing carts,6128 S KOMENSKY AVE,60629.0,1150420.567,...,"(41.781766462792376, -87.72406987512217)",170316502002002,650200,0,17,31,58911,4.21,7653,98
6,1427,2/7/17,Completed,2/7/17,17-00684661,Sanitation Code Violation,Overflowing carts,6110 S KOMENSKY AVE,60629.0,1150417.167,...,"(41.7820971998701, -87.72407921030418)",170316502002002,650200,0,17,31,58911,4.21,7653,98
7,2222,2/22/17,Completed,2/23/17,17-01015671,Sanitation Code Violation,Garbage in alley,6300 S PULASKI RD,60629.0,1150785.840,...,"(41.77865823677643, -87.72276009557498)",170316502002020,650200,0,17,31,58911,4.21,7653,98
8,3995,3/31/17,Open,NaN,17-01837889,Sanitation Code Violation,Dog feces in yard,6135 S KOLMAR AVE,60629.0,1147091.262,...,"(41.78150409271652, -87.73627913639721)",170316502006004,650200,0,17,31,58911,4.21,7653,98
9,1,1/2/17,Completed,1/3/17,17-00015443,Sanitation Code Violation,Overflowing carts,2433 N ASHLAND AVE,60614.0,1165256.823,...,"(41.926120388059594, -87.66817909285737)",170310706002016,70600,1,17,31,172638,3.08,3218,42


In [33]:
vacant_demo = pd.merge(vacant_fips, track_data, on='Track')

In [38]:
sani_top = sani_demo['Track'].value_counts().to_frame().reset_index()
sani_top.columns = ['Track','Count']

In [52]:
sani_top_demo = pd.merge(sani_top, track_data, on='Track')
sani_top_demo[:10]

,Track,Count,Unnamed: 0,State,County,Mean Income,Family Size,Population,Black
0,160602,66,148,17,31,62135,3.47,3970,83
1,570200,34,70,17,31,62262,4.02,1845,13
2,570300,33,76,17,31,53746,4.29,5966,11
3,260800,30,284,17,31,49602,3.95,2453,2432
4,71100,29,55,17,31,147379,2.87,2861,112
5,20702,29,524,17,31,61255,4.13,7833,312
6,841100,28,78,17,31,39517,3.28,7673,345
7,681200,25,40,17,31,30088,3.80,2437,2241
8,720700,25,129,17,31,84496,3.29,3254,1947
9,838700,25,130,17,31,25322,4.15,4736,4396


In [50]:
vacant_top = vacant_demo['Track'].value_counts().to_frame().reset_index()
vacant_top.columns = ['Track','Count']

In [53]:
vacant_top_demo = pd.merge(vacant_top, track_data, on='Track')
vacant_top_demo[:10]

,Track,Count,Unnamed: 0,State,County,Mean Income,Family Size,Population,Black
0,491000,21,302,17,31,36487,4.55,4785,4743
1,510300,17,366,17,31,43183,3.80,5988,5807
2,420700,14,104,17,31,35488,3.97,3505,3360
3,530200,13,328,17,31,40212,4.51,4675,4644
4,681400,13,167,17,31,38358,3.87,2624,2624
5,490902,12,576,17,31,48829,3.50,5146,5045
6,671500,11,350,17,31,30586,3.91,3012,2672
7,470100,11,447,17,31,38552,4.42,2601,2601
8,460100,11,515,17,31,53531,3.86,3751,2903
9,231200,11,228,17,31,35507,3.86,5369,3096


In [54]:
sani_demo['Creation Date'] = pd.to_datetime(sani_demo['Creation Date'],infer_datetime_format=True)
sani_demo['Month'] = pd.DatetimeIndex(sani_demo['Creation Date']).month

In [57]:
vacant_demo['DATE SERVICE REQUEST WAS RECEIVED'] = pd.to_datetime(vacant_demo['DATE SERVICE REQUEST WAS RECEIVED'],infer_datetime_format=True)
vacant_demo['Month'] = pd.DatetimeIndex(vacant_demo['DATE SERVICE REQUEST WAS RECEIVED']).month

In [58]:
sani_by_month_by_track = pd.pivot_table(sani_demo, values='Service Request Number', index=['Month'], columns=['Track'], aggfunc=lambda x: len(x.unique()))

In [59]:
sani_by_month_by_track

Track,10100,10201,10202,10300,10501,10502,10503,10600,10701,10702,...,843000,843100,843200,843300,843400,843500,843600,843700,843800,843900
Month,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,1.0,NaN,3.0,1.0,NaN,1.0,6.0,2.0,...,1.0,NaN,NaN,1.0,NaN,1.0,NaN,3.0,NaN,2.0
2,4.0,3.0,NaN,1.0,1.0,NaN,NaN,2.0,1.0,1.0,...,NaN,1.0,2.0,NaN,NaN,3.0,1.0,1.0,4.0,2.0
3,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,1.0,NaN,NaN,1.0,3.0,1.0,NaN,2.0,2.0,3.0
4,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
